In this kernel, we will explore the complete workflow for the APTOS 2019 competition. We will go through:

1. Loading & Exploration: A quick overview of the dataset
2. Resize Images: We will resize both the training and test images to 224x224, so that it matches the ImageNet format.
3. Mixup & Data Generator: We show how to create a data generator that will perform random transformation to our datasets (flip vertically/horizontally, rotation, zooming). This will help our model generalize better to the data, since it is fairly small (only ~3000 images).
4. Quadratic Weighted Kappa: A thorough overview of the metric used for this competition, with an intuitive example. Check it out!
5. Model: We will use a DenseNet-121 pre-trained on ImageNet. We will finetune it using Adam for 15 epochs, and evaluate it on an unseen validation set.
6. Training & Evaluation: We take a look at the change in loss and QWK score through the epochs.

### Citations & Resources

* I had the idea of using mixup from [KeepLearning's ResNet50 baseline](https://www.kaggle.com/mathormad/aptos-resnet50-baseline). Since the implementation was in PyTorch, I instead used an [open-sourced keras implementation](https://github.com/yu4u/mixup-generator).
* The transfer learning procedure is mostly inspired from my [previous kernel for iWildCam](https://www.kaggle.com/xhlulu/densenet-transfer-learning-iwildcam-2019). The workflow was however heavily modified since then.
* Used similar [method as Abhishek](https://www.kaggle.com/abhishek/optimizer-for-quadratic-weighted-kappa) to find the optimal threshold.
* [Lex's kernel](https://www.kaggle.com/lextoumbourou/blindness-detection-resnet34-ordinal-targets) prompted me to try using Multilabel instead of multiclass classification, which slightly improved the kappa score.

In [1]:
import json
import math
import os
import pdb

import cv2
from PIL import Image
import numpy as np
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
from keras.models import Model
from keras.layers import *
from keras.applications import *
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import plot_model
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
from sklearn.preprocessing import StandardScaler
import scipy
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")


%matplotlib inline

Using TensorFlow backend.


# Loading & Exploration

In [2]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
train_meta = pd.read_csv('../data/train_meta.csv')
test_meta = pd.read_csv('../data/test_meta.csv')
print(train_df.shape)
print(test_df.shape)
print(train_meta.shape, test_meta.shape)

(3662, 2)
(1928, 1)
(3662, 8) (1928, 7)


In [3]:
train_meta.head()

,id_code,diagnosis,image_shape,image_size,height,width,width_height_ratio,width_height_added
0,000c1434d8d7,2,"(2136, 3216, 3)",3218676,2136,3216,0.664179,5352
1,001639a390f0,4,"(2136, 3216, 3)",2261129,2136,3216,0.664179,5352
2,0024cdab0c1e,1,"(1736, 2416, 3)",1882172,1736,2416,0.718543,4152
3,002c21358ce6,0,"(1050, 1050, 3)",975218,1050,1050,1.000000,2100
4,005b95c28852,0,"(1536, 2048, 3)",1819430,1536,2048,0.750000,3584


In [4]:
m_train = train_meta[['image_size', 'height', 'width', 'width_height_ratio', 'width_height_added']].values
m_test = test_meta[['image_size', 'height', 'width', 'width_height_ratio', 'width_height_added']].values

In [5]:
SS = StandardScaler()
SS.fit(m_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [6]:
m_train = SS.transform(m_train)
m_test = SS.transform(m_test)

In [7]:
m_train.shape, m_test.shape

((3662, 5), (1928, 5))

In [8]:
def preprocess_image(image_path, desired_size=224):
    im = Image.open(image_path)
    im = im.resize((desired_size, )*2, resample=Image.LANCZOS)
    return im

In [9]:
def load_ben_color(path, size, crop=False, sigmaX=10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if crop:
        image = crop_image_from_gray(image)
    image = cv2.resize(image, (size, size))
    image = cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
    return image

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [10]:
size = 256
N = train_df.shape[0]
x_train = np.empty((N, size, size, 3), dtype=np.uint8)

for i, image_id in enumerate(tqdm(train_df['id_code'])):
    x_train[i, :, :, :] = np.load(
        f'../data/train_images/npy_bengrahm_color/{image_id}.npy'
    )

100%|██████████| 3662/3662 [00:06<00:00, 539.69it/s]


In [11]:
N = test_df.shape[0]
x_test = np.empty((N, size, size, 3), dtype=np.uint8)

for i, image_id in enumerate(tqdm(test_df['id_code'])):
    x_test[i, :, :, :] = load_ben_color(
        f'../data/test_images/{image_id}.png', size, crop=True
    )

100%|██████████| 1928/1928 [01:19<00:00, 24.34it/s]


In [12]:
#!mkdir ../data/npy_files
#np.save('../data/npy_files/x_train.npy', x_train)

In [13]:
#x_train = np.load('../data/npy_files/x_train.npy')
#x_test = np.load('../data/npy_files/x_test.npy')

In [14]:
y_train = pd.get_dummies(train_df['diagnosis']).values
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(3662, 256, 256, 3)
(3662, 5)
(1928, 256, 256, 3)


## Creating multilabels

Instead of predicting a single label, we will change our target to be a multilabel problem; i.e., if the target is a certain class, then it encompasses all the classes before it. E.g. encoding a class 4 retinopathy would usually be `[0, 0, 0, 1]`, but in our case we will predict `[1, 1, 1, 1]`. For more details, please check out [Lex's kernel](https://www.kaggle.com/lextoumbourou/blindness-detection-resnet34-ordinal-targets).

In [15]:
y_train_multi = np.empty(y_train.shape, dtype=y_train.dtype)
y_train_multi[:, 4] = y_train[:, 4]

for i in range(3, -1, -1):
    y_train_multi[:, i] = np.logical_or(y_train[:, i], y_train_multi[:, i+1])

print("Original y_train:", y_train.sum(axis=0))
print("Multilabel version:", y_train_multi.sum(axis=0))

Original y_train: [1805  370  999  193  295]
Multilabel version: [3662 1857 1487  488  295]


In [16]:
x_train.shape, y_train_multi.shape

((3662, 256, 256, 3), (3662, 5))

In [17]:
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train_multi, 
    test_size=0.15, 
    random_state=2019
)
m_train, m_val, = train_test_split(
    m_train, 
    test_size=0.15, 
    random_state=2019
)


In [18]:
x_train = x_train.astype('float32')
x_train.dtype

dtype('float32')

In [19]:
m_train.shape, m_val.shape, x_train.shape, x_val.shape

((3112, 5), (550, 5), (3112, 256, 256, 3), (550, 256, 256, 3))

# Mixup & Data Generator

In [20]:
class MixupGenerator():
    def __init__(self, X_train, y_train, batch_size=32, alpha=0.2, shuffle=True, datagen=None):
        self.X_train = X_train
        self.y_train = y_train
        self.batch_size = batch_size
        self.alpha = alpha
        self.shuffle = shuffle
        self.sample_num = len(X_train)
        self.datagen = datagen

    def __call__(self):
        while True:
            indexes = self.__get_exploration_order()
            itr_num = int(len(indexes) // (self.batch_size * 2))

            for i in range(itr_num):
                batch_ids = indexes[i * self.batch_size * 2:(i + 1) * self.batch_size * 2]
                X, y = self.__data_generation(batch_ids)

                yield X, y

    def __get_exploration_order(self):
        indexes = np.arange(self.sample_num)

        if self.shuffle:
            np.random.shuffle(indexes)

        return indexes

    def __data_generation(self, batch_ids):
        _, h, w, c = self.X_train.shape
        l = np.random.beta(self.alpha, self.alpha, self.batch_size)
        X_l = l.reshape(self.batch_size, 1, 1, 1)
        y_l = l.reshape(self.batch_size, 1)

        X1 = self.X_train[batch_ids[:self.batch_size]]
        X2 = self.X_train[batch_ids[self.batch_size:]]
        X = X1 * X_l + X2 * (1 - X_l)

        if self.datagen:
            for i in range(self.batch_size):
                X[i] = self.datagen.random_transform(X[i])
                X[i] = self.datagen.standardize(X[i])

        if isinstance(self.y_train, list):
            y = []

            for y_train_ in self.y_train:
                y1 = y_train_[batch_ids[:self.batch_size]]
                y2 = y_train_[batch_ids[self.batch_size:]]
                y.append(y1 * y_l + y2 * (1 - y_l))
        else:
            y1 = self.y_train[batch_ids[:self.batch_size]]
            y2 = self.y_train[batch_ids[self.batch_size:]]
            y = y1 * y_l + y2 * (1 - y_l)

        return X, y

In [21]:
class DualGenerator():
    def __init__(self, X_train, m_train, y_train, batch_size=32, shuffle=True, datagen=None):
        self.m_train = m_train
        self.X_train = X_train
        self.y_train = y_train
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.sample_num = len(X_train)
        self.datagen = datagen

    def __call__(self):
        while True:
            indexes = self.__get_exploration_order()
            itr_num = int(len(indexes) // (self.batch_size))

            for i in range(itr_num):
                batch_ids = indexes[i * self.batch_size:(i + 1) * self.batch_size]
                X, y = self.__data_generation(batch_ids)
                m_X = self.__meta_generation(batch_ids)
                yield [X, m_X], y

    def __get_exploration_order(self):
        indexes = np.arange(self.sample_num)

        if self.shuffle:
            np.random.shuffle(indexes)

        return indexes

    def __meta_generation(self, batch_ids):
        return self.m_train[batch_ids]
    
    def __data_generation(self, batch_ids):
        X = self.X_train[batch_ids]
        if self.datagen:
            for i in range(self.batch_size):
                X[i] = self.datagen.random_transform(X[i])
                X[i] = self.datagen.standardize(X[i])
        y = self.y_train[batch_ids]

        return X, y

In [22]:
BATCH_SIZE = 16

def create_datagen():
    return ImageDataGenerator(
        zoom_range=0.15,  # set range for random zoom
        # set mode for filling points outside the input boundaries
        fill_mode='constant',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,  # randomly flip images
        rescale=1.0/255.0,
    )

# Using original generator
#data_generator = create_datagen().flow(x_train, y_train, batch_size=BATCH_SIZE)
data_generator = DualGenerator(x_train, m_train, y_train, batch_size=BATCH_SIZE, datagen=create_datagen())() 
# Using Mixup
#mixup_generator = MixupGenerator(x_train, y_train, batch_size=BATCH_SIZE, alpha=0.2, datagen=create_datagen())()

In [23]:
gen = create_datagen()

In [24]:
x = next(iter(data_generator))

In [25]:
len(x[0])
x[0][0].shape, x[0][1]

((16, 256, 256, 3),
 array([[ 1.8783525 ,  0.79465274,  0.64785739, -0.31804108,  0.70756588],
        [-0.72626487, -0.87880537, -1.09160508,  1.6082367 , -1.01625675],
        [-1.11564773, -1.92932423, -1.55531094, -0.37002001, -1.7069134 ],
        [-0.10726543,  0.38550329,  0.45332713, -0.61893973,  0.42989372],
        [-0.2358391 ,  0.38550329,  0.45332713, -0.61893973,  0.42989372],
        [ 1.56000595,  0.79465274,  0.64785739, -0.31804108,  0.70756588],
        [-0.23749156,  0.01690018,  0.03712284, -0.37002001,  0.02959476],
        [-1.14855411, -1.68236015, -1.35286375, -0.37243546, -1.48632612],
        [-0.75846134, -0.87880537, -1.09160508,  1.6082367 , -1.01625675],
        [ 1.87998625,  0.79465274,  0.64785739, -0.31804108,  0.70756588],
        [-1.15677843, -1.92932423, -1.55531094, -0.37002001, -1.7069134 ],
        [-0.28311265,  0.01690018,  0.03712284, -0.37002001,  0.02959476],
        [ 0.10502746,  0.38550329,  0.45332713, -0.61893973,  0.42989372],
     

In [26]:
np.max(x[0][0])

1.0

### Creating keras callback for QWK

In [27]:
class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_kappas = []

    def on_epoch_end(self, epoch, logs={}):
#         pdb.set_trace()
        X_val, m_val, y_val = self.validation_data[:3]
        y_val = y_val.sum(axis=1) - 1
        
        y_pred = self.model.predict([X_val, m_val]) > 0.5 # <<<<<<<<<<<<<<<<<<<
        y_pred = y_pred.astype(int).sum(axis=1) - 1

        _val_kappa = cohen_kappa_score(
            y_val,
            y_pred, 
            weights='quadratic'
        )

        self.val_kappas.append(_val_kappa)

        print(f"val_kappa: {_val_kappa:.4f}")
        
        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved. Saving model.")
            self.model.save('model.h5')

        return

# Model: DenseNet-121

In [28]:
# xception = Xception(
#     include_top=False,
#    input_shape=(224, 224, 3)
# ) 

In [29]:
del model

NameError: name 'model' is not defined

In [ ]:
size=256
densenet = DenseNet121(
    include_top=False,
    input_shape=(224,224,3)
)
img_input = Input(shape=(size, size, 3, ))
img_out = densenet(img_input)
img_out = GlobalAveragePooling2D()(img_out)
img_out = Dropout(0.3)(img_out)
img_out = Dense(1024, activation='relu')(img_out)

meta_input = Input(shape=(5, ))
meta_out = Dense(256, activation="relu")(meta_input)
meta_out = Dropout(0.5)(meta_out)
meta_out = Dense(256, activation="relu")(meta_out)

merged = Concatenate()([img_out, meta_out])
output = Dropout(0.3)(merged)
output = Dense(5, activation="sigmoid")(output)

model = Model(inputs=[img_input, meta_input], output=output)
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(lr=0.00005),
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
#plot_model(model, to_file="metamodel.png")

# Training & Evaluation

In [ ]:
kappa_metrics = Metrics()

history = model.fit_generator(
    data_generator,
    steps_per_epoch=x_train.shape[0] / BATCH_SIZE,
    epochs=10,
    validation_data=([x_val, m_val], y_val),
    callbacks=[kappa_metrics]
)

In [ ]:
with open('xception_history.json', 'w') as f:
    json.dump(history.history, f)

In [ ]:

history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['acc', 'val_acc']].plot()

In [ ]:
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['acc', 'val_acc']].plot()

In [ ]:
plt.plot(kappa_metrics.val_kappas)

# LOAD test set with  bengrahm preprocessing ffs, before final prediction 

In [ ]:
#model.load_weights('model_bengrahm.h5')
y_val_pred = model.predict(x_val)

def compute_score_inv(threshold):
    y1 = y_val_pred > threshold
    y1 = y1.astype(int).sum(axis=1) - 1
    y2 = y_val.sum(axis=1) - 1
    score = cohen_kappa_score(y1, y2, weights='quadratic')
    
    return 1 - score

simplex = scipy.optimize.minimize(
    compute_score_inv, 0.5, method='nelder-mead'
)

best_threshold = simplex['x'][0]
best_threshold

In [ ]:
#best_threshold = 0.45
best_threshold

## Submit

In [ ]:
for th in [0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55]: 
    y_test = model.predict(x_test) > th
    y_test = y_test.astype(int).sum(axis=1) - 1
    print(th, np.unique(y_test, return_counts=True))
# test_df['diagnosis'] = y_test
# test_df.to_csv('submission.csv',index=False)


In [ ]:
np.unique(y_test, return_counts=True)

In [ ]:
np.unique(y_test, return_counts=True)